***
### Metodos

Extraido do TFS os changesets:<br>
```python
tf hist /noprompt /recursive /format:detailed /version:D"1/1/2015"~D"31/12/2015" $/fPJ/PJ/build/release > 
```
Verificar as alterações:<br>
```python
"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" difference $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Cliente/uspjConfirmacaoPadrao.pas  /format:unix /noprompt /version:C555472~C555471 /ignorespace      
```
Pegar arquivo na versão:<br>
```python
"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" get $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas /version:C553471 /force /noprompt
```

** Utilizado arquivo TFS.ipynb para conversão em CSV e Changesets.ipynb.

In [1]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt

%matplotlib inline

#arquivos = ['release_changeset_20171.csv']
arquivos = ['C:\\temp\\main_changeset.csv']
TFS = '"D:\\\TFS\\\Microsoft Visual Studio 10.0\\\Common7\\\IDE\\\\tf.exe"'

In [42]:
dfChangesets = pd.read_csv(arquivos[0], encoding='ISO-8859-1', delimiter=';', header=None,
                           names= ["Salt", "Changeset", "Desenv", "Data", "Descricao", "File", "Filename"],
                           chunksize=20000, iterator=True)

In [41]:
for chunk in dfChangesets:
    print(chunk.shape)

In [36]:
dfChangesets.shape

AttributeError: 'TextFileReader' object has no attribute 'shape'

In [ ]:
#for index, row in dfChangesets[(dfChangesets.Salt == '247275/4')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == '260601/1')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == '241736/1')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == 'Defect 120562')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == '1/1')].iterrows():
chunk_iter = 0
for chunk in dfChangesets:
    geral = {'NOME':[],'SALT':[],'CHANGESET':[],'DESENV':[],'ARQUIVO':[]}
    metodos = []
    for index, row in chunk.iterrows():
        if row['File'][-3:] != 'pas':
            continue
        #print (row)
        diferenca = diff(row)
        #print (diferenca)
        linhas = line(diferenca)
        #print (linhas)
        arquivo = get(row)
        #print (arquivo[-3:])
        if os.path.isfile(arquivo):
            if arquivo[-3:] == 'pas':
                for linha in linhas:
                    nome = metodo(arquivo, int(linha))
                    if nome != '':
                        if not (nome in metodos):
                            metodos.append([nome, arquivo])
                        #print (nome)
        for nome, arquivo in metodos:
            #print (nome)
            geral['NOME'].append(nome)
            geral['SALT'].append(row["Salt"])
            geral['CHANGESET'].append(row["Changeset"])
            geral['DESENV'].append(row["Desenv"])
            geral['ARQUIVO'].append(arquivo)

    data_frame = pd.DataFrame.from_dict(geral, dtype=object)
    data_frame.to_csv(path_or_buf='nome'+str(chunk_iter)+'.csv', index=False)
    chunk_iter += 1
    #print (geral

linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['All files are up to date.']
linhas: ['Al

In [270]:
dfChangesets[(dfChangesets.Salt == '241736/1')]

,Salt,Changeset,Desenv,Data,Descricao,File,Filename
143593,241736/1,521073,douglas.zuliani,8/6/2017,SALT 241736/1 - Corrigido problema ao lançar ...,uspjProcessomvServ.pas,/fPJ/PJ/build/main/pjDev/src/pj/Servidor/uspjP...


In [ ]:
df = pd.DataFrame(geral)
df.to_csv('C:\\temp\\geral.csv', index=False, header=False, encoding ='ISO-8859-1', sep=';')

In [ ]:
#File: D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Cliente\uspjConfirmacaoPadrao.pas
#===================================================================
#150,151c150
#<   // 26/12/2017 - Wilian.Ivo - Salt: 262749/1
#<   if not (nErroLogin in [nLoginOK, nLoginSenhaPrestesAExpirar]) then
#---
#>   if nErroLogin > 0 then
#===================================================================

def diff(row):
    comando = TFS + ' difference ' + '$' + row['Filename'] + ' /format:unix /noprompt '
    comando += '/version:' + 'C' + str(row['Changeset']) + '~' + 'C' + str(int(str(row['Changeset']).replace('C', '')) - 1) + ' /ignorespace'
    #print (comando)
    pipe = os.popen(comando)
    retorno = pipe.read()
    pipe.close()
    #print (a)
    return retorno

teste = {'Filename':'/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Cliente/uspjConfirmacaoPadrao.pas', 'Changeset':'555472'}
print (diff(teste))

teste = {'Filename':'/fPJ/PJ/build/release/4.0.18/USP/pjDev/src/3rd_party/edtDev/src/Cliente/uedtEditor5.pas', 'Changeset':'551248'}
print (diff(teste))


File: D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Cliente\uspjConfirmacaoPadrao.pas
150,151c150
<   // 26/12/2017 - Wilian.Ivo - Salt: 262749/1
<   if not (nErroLogin in [nLoginOK, nLoginSenhaPrestesAExpirar]) then
---
>   if nErroLogin > 0 then

File: D:\UNJ\fPJ\PJ\build\release\4.0.18\USP\pjDev\src\3rd_party\edtDev\src\Cliente\uedtEditor5.pas
1411,1412d1410
<     // 07/12/2017 - Anderson.Albuquerque - SALT 260601/1
<     procedure PassarParametrosParaEnvioDeEmail(oForm: TspForm); virtual;
15266,15267c15264
<     // 07/12/2017 - Anderson.Albuquerque - SALT 260601/1
<     PassarParametrosParaEnvioDeEmail(oForm);
---
>     oForm.PassaParametro(sPRM_ADD_ARQS_EMAIL, FPathArquivoEmail);
21121,21126d21117
< // 07/12/2017 - Anderson.Albuquerque - SALT 260601/1
< procedure TfedtEditor5.PassarParametrosParaEnvioDeEmail(oForm: TspForm);
< begin
<   oForm.PassaParametro(sPRM_ADD_ARQS_EMAIL, FPathArquivoEmail);
< end;
< 



In [2]:
pipe = os.popen('"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" difference $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas /format:unix /noprompt /version:C553471~C553470 /ignorespace')
a = pipe.read()
pipe.close()
print (a)


File: D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Servidor\uspjSelectFluxoObjetoBasico.pas
544a545
>     wflSelectPrincipal.AddColuna('p.flLitispendencia as cc_flLitispendencia');
546d546
<     //13/12/2017 - Wilian.Ivo - Salt: 255004/2
548,559c548,552
<       FSQL.Decode(FSQL.Value('pSup.flLitispendencia', 'p.flLitispendencia'),
<       [QuotedStr(cLitisPendente), QuotedStr(sLITIS_PENDENTE), QuotedStr(cLitisSuspeita),
<       QuotedStr(sLITIS_SUSPEITA), QuotedStr(cLitisConfirmada), QuotedStr(sLITIS_CONFIRMADA),
<       QuotedStr(cLitisConexao), QuotedStr(sLITIS_CONEXAO), QuotedStr(cLitisDescartada),
<       QuotedStr(sLITIS_DESCARTADA), QuotedStr(sLITIS_INEXISTENTE)]) + ' as cc_deLitispendencia');
< 
<     //26/12/2017 - Wilian.Ivo - Salt: 255004/2
<     if (oParametrosConsultaObjetoFila.TestarCampoNaProjecao('cc_deLitispendencia')) then
<       wflSelectPrincipal.AddColuna(FSQL.Value('pSup.flLitispendencia', 'p.flLitispendencia') +
<         ' as cc_flLitispendencia')
<   

In [45]:
#D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Servidor:
#Replacing uspjSelectFluxoObjetoBasico.pas

def get(row):
    #pipe = os.popen('"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" get $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas /version:C553471 /force /noprompt')
    comando = TFS + ' get $' + row['Filename'] + ' /version:' + 'C'
    comando += str(row['Changeset']) + ' /force /noprompt'
    #print(comando)
    pipe = os.popen(comando)
    retorno = pipe.read()
    pipe.close()
    #print (retorno)
    linhas = retorno.splitlines()
    #print(len(linhas))
    caminho = ''
    arquivo = ''
    if linhas[0] != 'All files are up to date.':
        try:
            caminho = linhas[0][:len(linhas[0])-1]
            #caminho = caminho.replace('\\', '\\\\')
            arquivo = linhas[1].replace('Replacing ', '').replace('Getting ', '')
        except:
            print ('linhas:',linhas)
    return caminho + '\\' + arquivo

teste = {'Filename':'/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas', 'Changeset':'553471'}
get(teste)

'D:\\UNJ\\fPJ\\PJ\\build\\release\\4.20\\producao\\pjDev\\src\\pj\\Servidor\\uspjSelectFluxoObjetoBasico.pas'

In [44]:
def line(diff):
    retorno = []
    for linha in diff.splitlines():
        if linha[0].isdigit():
            registro = linha.split(',')[0]
            if registro.isdigit():
                retorno.append(registro)
            else:
                registro = linha.split('c')[0]
                if registro.isdigit():
                    retorno.append(registro)
                else:
                    registro = linha.split('d')[0]
                    if registro.isdigit():
                        retorno.append(registro)
    return retorno


In [31]:
def metodo(arquivo, linha):
    #f = open('D:\\UNJ\\fPJ\\PJ\\build\\release\\4.20\\producao\\pjDev\\src\\pj\\Servidor\\uspjSelectFluxoObjetoBasico.pas')
    f = open(arquivo)
    lines = f.readlines()
    if linha > len(lines):
        print ('arquivo:', arquivo)
        metodo = ''
    else:
        while (linha > 0):
            texto = ''
            try:
                texto = lines[linha]
            except:
                #print ('lines:', lines)
                texto = ''
                metodo = ''
            if re.match('function.*\..*\;', texto) or re.match('procedure.*\..*\;', texto): 
                metodo = texto
                break
            linha -= 1
        else:
            metodo = ''
    return metodo.replace('\n', '')

metodo('D:\\UNJ\\fPJ\\PJ\\build\\release\\4.20\\producao\\pjDev\\src\\pj\\Servidor\\uspjSelectFluxoObjetoBasico.pas', 800)

'procedure TspjSelectFluxoObjetoBasico.AdicionarCondicoes;'